In [ ]:
!pip install fsspec==2024.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers datasets peft accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00


In [ ]:
import torch
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_load_path = "/content/drive/MyDrive/Creativity/Qwen2.5_7b_fine_tuned_model_wiki"

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_16_FT_wiki_narrative.xlsx'
df = pd.read_excel(file_path, sheet_name ='Sheet1')

In [ ]:
# Load the model and tokenizer
# model_name = "Qwen/Qwen2.5-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_load_path, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_load_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
# Print model architecture to find correct target module names
# for name, module in model.named_modules():
#     print(name, module)

In [ ]:
pad_token_id = tokenizer.eos_token_id

In [ ]:
# Prepare LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,                 # Low-rank adaptation parameter
    lora_alpha=32,            # Scaling factor
    lora_dropout=0.1,          # Dropout to prevent overfitting
    target_modules=[
        f"layers.{i}.self_attn.q_proj" for i in range(20, 28)
    ] + [
        f"layers.{i}.self_attn.k_proj" for i in range(20, 28)
    ] + [
        f"layers.{i}.self_attn.v_proj" for i in range(20, 28)
    ],
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 1,966,080 || all params: 7,617,582,592 || trainable%: 0.0258


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# dataset = load_dataset("wikitext", "wikitext-2-raw-v1",trust_remote_code=True)

In [ ]:
# # Tokenize the dataset
# def tokenize_function(examples):
#     return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)

# tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# # Add labels to the tokenized dataset
# tokenized_dataset = tokenized_dataset.map(lambda examples: {'labels': examples['input_ids']}, batched=True)
# tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
narrativeqa_dataset = load_dataset("narrativeqa", trust_remote_code=True)

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

train-00001-of-00024.parquet:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

train-00003-of-00024.parquet:   0%|          | 0.00/27.2M [00:00<?, ?B/s]

...

validation-00002-of-00003.parquet:   0%|          | 0.00/68.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32747 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10557 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3461 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
def preprocess_narrativeqa(examples):
    # Extract 'text' from the nested 'document' and 'question' fields
    documents = [doc.get("text", "") if isinstance(doc, dict) else doc for doc in examples["document"]]
    questions = [q.get("text", "") if isinstance(q, dict) else q for q in examples["question"]]

    # Combine context and question for input
    inputs = [f"Context: {doc}\nQuestion: {q}" for doc, q in zip(documents, questions)]
    targets = [ans[0]["text"] if ans else "" for ans in examples["answers"]]

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, max_length=5120, truncation=False, padding="max_length")
    # with tokenizer.as_target_tokenizer(): # Use as_target_tokenizer for labels
    #     labels = tokenizer(text_target=targets, max_length=128, truncation=True, padding="max_length").input_ids

    labels = tokenizer(
        text_target=targets,
        max_length=5120,
        truncation=False,
        padding="max_length"
    ).input_ids

    # Replace padding token id with -100 for loss calculation
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in target] for target in labels]

    # Ensure labels length matches input_ids length
    # This assumes 'input_ids' key is used in model_inputs
    # for i in range(len(labels)):
    #     if len(labels[i]) != len(model_inputs['input_ids'][i]):
    #         # Adjust labels length to match input_ids
    #         if len(labels[i]) < len(model_inputs['input_ids'][i]):
    #             labels[i] += [-100] * (len(model_inputs['input_ids'][i]) - len(labels[i]))
    #         else:
    #             labels[i] = labels[i][:len(model_inputs['input_ids'][i])]

    model_inputs["labels"] = labels
    return model_inputs

In [ ]:
tokenized_narrativeqa = narrativeqa_dataset.map(
    preprocess_narrativeqa, batched=True, remove_columns=narrativeqa_dataset["train"].column_names
)

Map:   0%|          | 0/32747 [00:00<?, ? examples/s]

Map:   0%|          | 0/10557 [00:00<?, ? examples/s]

Map:   0%|          | 0/3461 [00:00<?, ? examples/s]

In [ ]:
# Set training arguments
training_args = TrainingArguments(
    output_dir="./Qwen2.5-7b--finetuned",
    run_name="Qwen2.5-7b-run",
    eval_strategy="steps",
    eval_steps=500,
    # logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.01,
    warmup_ratio=0.01,
    lr_scheduler_type="constant",
    save_strategy="steps",
    optim="paged_adamw_8bit",
    max_grad_norm=1.0,
    save_steps=200,
    save_total_limit=2,
    logging_dir="./logs-wikitext",
    fp16=True,
    # remove_unused_columns=False,
    dataloader_num_workers=4
)

tokenized_narrativeqa_train = tokenized_narrativeqa["train"]
tokenized_narrativeqa_validation = tokenized_narrativeqa["validation"]

from transformers import DataCollatorForSeq2Seq
# Define a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_narrativeqa_train,
    eval_dataset=tokenized_narrativeqa_validation,
    data_collator=data_collator
)

# Fine-tune the model with LoRA
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,8.887600
1000,7.055600


TrainOutput(global_step=1023, training_loss=7.946704808567976, metrics={'train_runtime': 2210.7664, 'train_samples_per_second': 14.813, 'train_steps_per_second': 0.463, 'total_flos': 7.112544756663583e+17, 'train_loss': 7.946704808567976, 'epoch': 0.9996335654085746})

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def generate_response(prompt, n=1 ,max_length=300):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = (input_ids != pad_token_id).long().to(device)
    responses = []
    for _ in range(n):
        with torch.no_grad():
            output = model.generate(input_ids,
                         attention_mask=attention_mask,
                         max_length=max_length,
                         do_sample=True,
                         top_k=50,
                         top_p=0.9,
                         temperature=1.0,
                         pad_token_id=pad_token_id)
            response = tokenizer.decode(output[0], skip_special_tokens=True)
            response = response[len(prompt):].strip()
        responses.append(response)
    return responses

In [ ]:
print(df.columns)

Index(['Prompts', 'Response_1', 'Response_2', 'Response_3', 'Functionality',
       'Fluency', 'Flexibility', 'Originality', 'Elaboration', 'TTCT'],
      dtype='object')


In [ ]:
# Ensure the columns can hold string data by converting them to string type
df['Response_1'] = df['Response_1'].astype(str)
df['Response_2'] = df['Response_2'].astype(str)
df['Response_3'] = df['Response_3'].astype(str)

In [ ]:
# Generate responses for each prompt and add them to the DataFrame
for idx, row in df.iterrows():
    prompt = row['Prompts']
    responses = generate_response(prompt)
    df.at[idx, 'Response_1'] = responses[0]
    responses = generate_response(prompt)
    df.at[idx, 'Response_2'] = responses[0]
    responses = generate_response(prompt)
    df.at[idx, 'Response_3'] = responses[0]

In [ ]:
# Save the updated DataFrame back to Google Drive
updated_file_path = '/content/drive/MyDrive/Creativity/Qwen2.5_7b_16_FT_wiki_narrative.xlsx'
df.to_excel(updated_file_path, index=False, engine='openpyxl')